In [ ]:
!pip install pylzma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylzma: filename=pylzma-0.5.0-cp310-cp310-linux_x86_64.whl size=222321 sha256=f80bad65fcf4a6a49f623cabbaea106fb310a50fd788cdc38ab47e540ce4b87b
  Stored in directory: /root/.cache/pip/wheels/74/c9/02/91112815e838f544c1d46fda071241e454694579d022751d2b
Successfully built pylzma


In [ ]:
!pip install cuda-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 10.3 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cuda


In [ ]:
with open('/content/drive/MyDrive/wizard_of_oz.txt','r',encoding ='utf-8') as f:
  text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [ ]:
# Tokenizer
string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
# print(data[:100])

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
def get_batch(split):
  data = train_data if split == "train" else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  print(ix)
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  x, y = x.to(device), y.to(device)
  return x,y

x,y = get_batch("train")
print('inputs:')
print(x)
print('targets:')
print(y)

tensor([ 19180, 112247, 160754, 168739])
inputs:
tensor([[54,  1, 65, 54, 67, 57, 72, 56],
        [73, 68, 71, 78,  1, 62, 67,  1],
        [ 1, 63, 74, 72, 73, 62, 56, 58],
        [58, 65, 10, 72, 73, 74, 57, 57]], device='cuda:0')
targets:
tensor([[ 1, 65, 54, 67, 57, 72, 56, 54],
        [68, 71, 78,  1, 62, 67,  1, 73],
        [63, 74, 72, 73, 62, 56, 58,  9],
        [65, 10, 72, 73, 74, 57, 57, 58]], device='cuda:0')


In [ ]:
@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out


In [ ]:

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is', context,'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [ ]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, index, targets= None):
    logits = self.token_embeddings_table(index)
    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, index, max_new_tokens):
    # index is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      # get the predictions
      logits, loss = self.forward(index)
      # focus only on the last time step
      logits = logits[:,-1,:] # becomes (B,C)
      # apply softmax to get the probabilities
      probs = F.softmax(logits, dim=-1) # (B,C)
      # sample from the distribution
      index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
      # append sampled index to the running sequence
      index = torch.cat((index, index_next), dim = 1) # (B, T-1)

    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype= torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)





O3Q&]ghvZPWJ﻿pzIrR
9h6XSTGKa83ECl;s7bxYSgy yXSw;10_CoAMqDoAHLvCJ"!U8UI&P.Yz'Ze&?QQI-b(r697J!x*&57fY.?f.j[Je!9"u;bt_mC7q*guT0a0vVx?pq[WzKYI49
f2tt'_zPLWGOZ,:s(o(D
VuTc:0DzP﻿a05O﻿cyua(hQ(oQsS'TN_DKKY ]TTeJA1R2 Hs8:t*B1Ub-McLg1 J﻿WH;W 0?4)"c8!(fQ;8jhGVxgQ2FAO!N1,c5WhGfy﻿1wTEdOlV9
!NiuDS(z

"m0oqm7beSpub:GECgwwG,Ecq2TQS:PLH[lq7hRA_-4L9HI03Hp;g(uOrptM_8ftUY in(oVPC"-N89kv6Q4vGkuWzzM_mlrfV&LZPbt;7G1CaBi 3?QtBt6pV,n'j8[vQ]Tyi0(
rnmHiyc?gHdG,4vu)k0mr,﻿twDIY﻿x9l(0_uTZnWYxT0O-;XEAGzP*n6W._-uTwPRFA!﻿W[iyP_


In [ ]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(),lr = learning_rate)

for iter in range (max_iters):
  if iter % eval_iters == 0:
    losses = estimate_loss()
    print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

  # sample a batch of data
  xb, yb = get_batch('train')
  # evaluate the loss
  logits, loss = model.forward(xb, yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward(retain_graph = True)
  optimizer.step()

print(loss.item())

tensor([164958,  35458, 114895, 134515])
tensor([ 63273, 107729,  14382,   6668])
tensor([173840, 129599,  41399, 154785])
tensor([128907,   3039,  21384,  43914])
tensor([ 93654, 130221,   4281,  60196])
tensor([ 41054, 153739, 157448,  61990])
tensor([109611,   9079,  78278,   1195])
tensor([177956,  85400, 175500, 148632])
tensor([ 80526,  15316, 117551, 149955])
tensor([166449, 150959, 163076,  49304])
tensor([103570, 103351,  52373,  74648])
tensor([150368,   3210, 119802, 155956])
tensor([115196, 130930,  91255,  49212])
tensor([  7314,  76390,  77778, 107101])
tensor([127780, 140758,  34799,  49854])
tensor([159260, 175368,  84407, 142722])
tensor([157403,  37903,  48620,  32147])
tensor([ 97813, 160447,  89934,   1899])
tensor([123557,   4054,  16212,  20111])
tensor([105354,  65245, 184676,  53430])
tensor([ 45704,  55806,  39009, 148451])
tensor([111881, 171328,    713,  51721])
tensor([  6885,  43776,  63993, 131893])
tensor([ 16612, 145235, 132025,  77483])
tensor([167926, 

In [ ]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(generated_chars)


JCx5Y6e6k*j9"9!jWXk'F*gw6el!ov.]b*X!xrdZP *G1:V0CFNJfYkaKIcb!"mo)LnV4J6-D1IJf[
GBKre0A9&xO7_1hBA
_m0i7ttoGi_j7aA!O
c0?;?:P﻿o5..8h)e1j5Elrs,Hk83Ess3ExAu",t wdzZ1zu9wmF4'?l
zJYRIeGvMD]7X. Y2t,.'0!dGrESMqf23Q5(d*B HLj;tJ﻿!9oGu﻿p;5.t*9k'pFNWcPCUyPxzJg_3yP.l;5diUhj*78[0yP*]g4p*jm8ZCd8i
d.GIA,_).bEpDa8]Vu?fU(I44&1CS5b
V&AD8(UbjibtvL
;48fSJk!-gVxO7qNp96Zn(dY Hk0?XPM_8-7a87514[9GEQ&328FAdF2t!xrnDNx4c"Oz7pziyPdQw_CIlgaUA52nu:&iD]HzXTA*gIs3a4[6T?ItMEtrpB,bMA[W.:Y PbjDc(5JBSL' ,_&HC2T,Y&4[*6er*62wGWBx-c]Hq
